In [8]:
import torch
from torch import nn

# 一、汇聚层
# 汇聚（pooling）层具有双重目的:降低卷积层对位置的敏感性,同时降低对空间降采样表示的敏感性。(减轻卷积层对位置的过度敏感)
"""
汇聚窗口形状为p*q的汇聚层称为p*q汇聚层,汇聚操作称为p*q汇聚。
使用汇聚层的目的:当被检测目标平移后,经过汇聚,卷积层仍然能找到指定的模式
如果输入的形状为 h*w ,汇聚层的形状是 ph*pw ,那么经过汇聚的输出形状是 (h-ph+1)*(w-pw+1)
"""
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0]-p_h+1, X.shape[1]-p_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i:i+p_h, j:j+p_w].max()
            elif mode == 'avg':
                Y[i,j] = X[i:i+p_h, j:j+p_w].mean()
    return Y
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
print(f"输入X经过max汇聚层的输出:\n{pool2d(X, (2, 2))}")
print(f"输入X经过mean汇聚层的输出:\n{pool2d(X, (2, 2), mode='avg')}")

# 二、填充和步幅在汇聚层中的使用
# 汇聚层也可以用填充和步幅改变所需的输出形状
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4)) # 前两个维度:样本数 通道数 都是1
pool2d = nn.MaxPool2d(kernel_size=3) # 深度学习框架中汇聚层的默认步幅和汇聚窗口的大小相同 
print(f"使用默认步幅的汇聚结果:\n{pool2d(X)}")
pool2d = nn.MaxPool2d(kernel_size=(2,3), stride=(2,3), padding=(0,1)) # 自定义参数
print(f"使用自定义参数的汇聚结果:\n{pool2d(X)}")

# 三、多通道
"""
在处理多通道输入数据时，汇聚层在每个输入通道上单独运算，而不是像卷积层一样在通道上对输入进行汇总。
这意味着汇聚层的输出通道数与输入通道数相同。
"""
X = torch.concat((X,X+1), dim=1) # 在通道维度连接
pool2d = nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
print(f"多通道的汇聚结果:\n{pool2d(X)}")

输入X经过max汇聚层的输出:
tensor([[4., 5.],
        [7., 8.]])
输入X经过mean汇聚层的输出:
tensor([[2., 3.],
        [5., 6.]])
使用默认步幅的汇聚结果:
tensor([[[[10.]]]])
使用自定义参数的汇聚结果:
tensor([[[[ 5.,  7.],
          [13., 15.]]]])
多通道的汇聚结果:
tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])
